In [1]:
import time
from tqdm import tqdm
import pandas as pd
import authorization
import sys


In [2]:
# If your authentification script is not in the project directory
# append its folder to sys.path
sys.path.append("../spotify_api_web_app")

# Authorize and call access object "sp"
sp = authorization.authorize()

# Get all genres
genres = sp.recommendation_genre_seeds()


In [3]:
# Set number of recommendations per genre
n_recs = 100

# Initiate a dictionary with all the information you want to crawl
data_dict = {"id": [], "genre": [], "track_name": [], "artist_name": [],
             "valence": [], "energy": []}

In [4]:
################
## CRAWL DATA ##
################

# Get recs for every genre
for g in tqdm(genres):

    # Get n recommendations
    recs = sp.recommendations(genres=[g], limit=n_recs)
    # json-like string to dict
    recs = eval(recs.json().replace("null", "-999").replace("false",
                "False").replace("true", "True"))["tracks"]

    # Crawl data from each track
    for track in recs:
        # ID and Genre
        data_dict["id"].append(track["id"])
        data_dict["genre"].append(g)
        # Metadata
        track_meta = sp.track(track["id"])
        data_dict["track_name"].append(track_meta.name)
        data_dict["artist_name"].append(track_meta.album.artists[0].name)
        # Valence and energy
        track_features = sp.track_audio_features(track["id"])
        data_dict["valence"].append(track_features.valence)
        data_dict["energy"].append(track_features.energy)

        # Wait 0.2 seconds per track so that the api doesnt overheat
        time.sleep(0.2)


  4%|▍         | 5/126 [04:54<1:58:40, 58.85s/it]


ServiceUnavailable: Error in https://api.spotify.com/v1/tracks/00WCxlPajxvVmgOfVpcLGc:
503: Service unavailable


In [ ]:
# Store data in dataframe
df = pd.DataFrame(data_dict)

# Drop duplicates
df.drop_duplicates(subset="id", keep="first", inplace=True)
df.to_csv("valence_arousal_dataset.csv", index=False)